In [4]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 10
train_images_path = keras.utils.get_file('train-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-images-idx3-ubyte.gz')
train_labels_path = keras.utils.get_file('train-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-labels-idx1-ubyte.gz')
test_images_path = keras.utils.get_file('t10k-images-idx3-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-images-idx3-ubyte.gz')
test_labels_path = keras.utils.get_file('t10k-labels-idx1-ubyte.gz', 'https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/t10k-labels-idx1-ubyte.gz')

def load_mnist(images_path, labels_path):
    import os
    import gzip
    import numpy as np

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

#if K.image_data_format() == 'channels_first':
 #   x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  #  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
   # input_shape = (1, img_rows, img_cols)

    

input_shape = (img_rows, img_cols, 1)
x_train, y_train = load_mnist(train_images_path, train_labels_path)
x_test, y_test = load_mnist(test_images_path, test_labels_path)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')




x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save('my_mnist_keras_cnn_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 199s - loss: 0.6233 - acc: 0.7798 - val_loss: 0.3992 - val_acc: 0.8543
Epoch 2/10
60000/60000 [==============================] - 209s - loss: 0.4016 - acc: 0.8580 - val_loss: 0.3369 - val_acc: 0.8810
Epoch 3/10
60000/60000 [==============================] - 203s - loss: 0.3542 - acc: 0.8752 - val_loss: 0.3148 - val_acc: 0.8878
Epoch 4/10
60000/60000 [==============================] - 194s - loss: 0.3252 - acc: 0.8851 - val_loss: 0.2974 - val_acc: 0.8950
Epoch 5/10
60000/60000 [==============================] - 198s - loss: 0.3011 - acc: 0.8929 - val_loss: 0.2901 - val_acc: 0.8959
Epoch 6/10
60000/60000 [==============================] - 205s - loss: 0.2868 - acc: 0.8980 - val_loss: 0.2705 - val_acc: 0.9041
Epoch 7/10
60000/60000 [==============================] - 194s - loss: 0.2732 - acc: 0.9031 - val_loss: 0.2635 - val_acc: 0.9039
Epoch 8/10
60000/60000 [=======================